In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam

from keras.applications import imagenet_utils
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

In [ ]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
n_classes = len(np.unique(y_train))
y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)

X_train = X_train.astype('float32')/255.
X_test = X_test.astype('float32')/255.

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False)

In [ ]:
x = vgg_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
out = Dense(10, activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg_model.input, outputs=out)

In [ ]:
for layer in vgg_model.layers:
    layer.trainable = False

In [ ]:
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
callbacks = [EarlyStopping(monitor='val_acc', patience=5, verbose=0)]

In [ ]:
n_epochs = 50
batch_size = 512
history = model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_split=0.2, verbose=1, callbacks=callbacks)

In [ ]:
plt.plot(np.arange(len(history.history['acc'])), history.history['acc'], label='training')
plt.plot(np.arange(len(history.history['val_acc'])), history.history['val_acc'], label='validation')
plt.title('Accuracy')
plt.xlabel('batches')
plt.ylabel('accuracy ')
plt.legend(loc=0)
plt.show()